#### Prueba Teorica

Un modelo de aprendizaje es una herramiento que trabaja sobre datos para poder clasificarlos

- Datos estructurados: Aquellos datos que respresentan algún tipo de información (metadato)
- Datos no estructurados: Aquellos datos que no se sabe lo que representan (por ejemplo: texto, imagenes, etc)

- Aprendizaje supervisado: Modelo de aprendizaje en el cual se conoce la estructura que se quiere estudiar (existen clase de datos predefinidos) 
- Aprendizaje no supervisado: Modelo de aprendizaje en el cual no se conoce la estructura que se estudia (no existen clases predefinidas, algoritmos intentan encontrar las clases que existen)

- Labels son las clases (Etiqueta) de datos (por ejemplo: flor 1, flor 2, flor 3, etc)
- Features son las caracteristicas de cada clase (por ejemplo: tamaño de tallo, número de pétalos, etc)

- Precision: De las predicciones totales, cuales fueron correctas (En vertical en la matriz de confusión)
- Recall: Del total de datos reales, cuales fueron correctamente identificados (En horizontal en la matriz de confusión)

Pasos  a seguir para aplicar un modelo:
- Limpiar dataset
- Separar dataset (entrenamiento, validación, prueba)
- Definir la variables descriptivas y la que queremos identificar
- Entrenar a traves algoritmo de aprendizaje (Aprender un modelo que clasifica los datos)
- Luego se valida el modelo utilizando una parte del dataset
- Se comparan los resultados del modelo con el dataset de validación real (recall y presicion, observando la matriz de confusión). Un error tipico es sobreaprender datos, para mejorar un modelo, el resultado es un modelo que funciona bien para los datos similares a lo que se usaron para aprender
- Se hace prueba con la última parte del dataset y para hacer comparaciones con resultado de la validación

- Regresion lineal: Modelo de aprendizaje para predecir valores continuos. Aprende los parametros (w) de la ecuación
- Regresión logistica: Modelo de aprendizaje para predecir valores binarios (llueve/no llueves, 1/0, vivo/muerto) aprende los parametros (w) de la ecuación y lo encapsula en un función logistica. Los parametros de la función se inicializan de forma aleatoria, se actualizan utilizando función de perdida (gradient descent)


$$
x=w_o + w_1 x_1 + w_2 x_2 ... \\ \\
P(x)=\frac{1}{1+\exp^{-x}}
$$


Función de perdida nos retorna un valor para determinar que tan correcto es un modelo para predecir datos. Cuando la función de perdida tiene pendiente cero (mínimo local), quiere decir que en ese punto es óptima la función de perdida

In [54]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

In [65]:
df = pd.read_csv("meteo.csv")
print("Dataset size")
print("Rows {} Columns {}".format(df.shape[0], df.shape[1]))

Dataset size
Rows 142193 Columns 24


In [66]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [67]:
print("Columns and data types")
pd.DataFrame(df.dtypes).rename(columns = {0:'dtype'}) #tipos de datos

Columns and data types


,dtype
Date,object
Location,object
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Evaporation,float64
Sunshine,float64
WindGustDir,object
WindGustSpeed,float64
WindDir9am,object


In [68]:
df.isnull().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
dtype: int64

In [69]:
df = df.drop(['Date','Location','Evaporation','Sunshine','WindGustDir','WindGustSpeed','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm',
             'Pressure9am','Pressure3pm','Cloud9am','Cloud3pm','RISK_MM'], 1)

In [70]:
df.isnull().sum()

MinTemp          637
MaxTemp          322
Rainfall        1406
Humidity9am     1774
Humidity3pm     3610
Temp9am          904
Temp3pm         2726
RainToday       1406
RainTomorrow       0
dtype: int64

In [71]:
MinTempMean=np.nanmean(df['MinTemp'])
MaxTempMean=np.nanmean(df['MaxTemp'])
RainfallMean=np.nanmean(df['Rainfall'])
Humidity9amMean=np.nanmean(df['Humidity9am'])
Humidity3pmMean=np.nanmean(df['Humidity3pm'])
Temp9amMean=np.nanmean(df['Temp9am'])
Temp3pmMean=np.nanmean(df['Temp3pm'])

In [72]:
def approx_MinTemp(fact):
    if pd.isnull(fact):
        return MinTempMean
    else:
        return fact

In [73]:
def approx_MaxTemp(fact):
    if pd.isnull(fact):
        return MaxTempMean
    else:
        return fact

In [74]:
def approx_Rainfall(fact):
    if pd.isnull(fact):
        return RainfallMean
    else:
        return fact

In [75]:
def approx_Humidity9am(fact):
    if pd.isnull(fact):
        return Humidity9amMean
    else:
        return fact

In [76]:
def approx_Humidity3pm(fact):
    if pd.isnull(fact):
        return Humidity3pmMean
    else:
        return fact

In [77]:
def approx_Temp9am(fact):
    if pd.isnull(fact):
        return Temp9amMean
    else:
        return fact

In [78]:
def approx_Temp3pm(fact):
    if pd.isnull(fact):
        return Temp3pmMean
    else:
        return fact

In [79]:
df['MinTemp'] = df['MinTemp'].apply(approx_MinTemp)
df['MaxTemp'] = df['MaxTemp'].apply(approx_MaxTemp)
df['Rainfall'] = df['Rainfall'].apply(approx_Rainfall)
df['Humidity9am'] = df['Humidity9am'].apply(approx_Humidity9am)
df['Humidity3pm'] = df['Humidity3pm'].apply(approx_Humidity3pm)
df['Temp9am'] = df['Temp9am'].apply(approx_Temp9am)
df['Temp3pm'] = df['Temp3pm'].apply(approx_Temp3pm)
df.isnull().sum()

MinTemp            0
MaxTemp            0
Rainfall           0
Humidity9am        0
Humidity3pm        0
Temp9am            0
Temp3pm            0
RainToday       1406
RainTomorrow       0
dtype: int64

In [80]:
df.dropna(inplace=True)
df.isnull().sum()

MinTemp         0
MaxTemp         0
Rainfall        0
Humidity9am     0
Humidity3pm     0
Temp9am         0
Temp3pm         0
RainToday       0
RainTomorrow    0
dtype: int64

In [81]:
pd.DataFrame(df.dtypes).rename(columns = {0:'dtype'}) #tipos de datos

,dtype
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Humidity9am,float64
Humidity3pm,float64
Temp9am,float64
Temp3pm,float64
RainToday,object
RainTomorrow,object


In [82]:
RainToday = pd.get_dummies(df['RainToday'],drop_first=True)
df.drop(['RainToday'],axis=1,inplace=True)
df=pd.concat([df,RainToday],axis=1)

names = df.columns.tolist()
names[names.index('Yes')] = 'RainToday'
df.columns = names

In [83]:
RainTomorrow = pd.get_dummies(df['RainTomorrow'],drop_first=True)
df.drop(['RainTomorrow'],axis=1,inplace=True)
df=pd.concat([df,RainTomorrow],axis=1)

names = df.columns.tolist()
names[names.index('Yes')] = 'RainTomorrow'
df.columns = names

In [84]:
df.head()

,MinTemp,MaxTemp,Rainfall,Humidity9am,Humidity3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,13.4,22.9,0.6,71.0,22.0,16.9,21.8,0,0
1,7.4,25.1,0.0,44.0,25.0,17.2,24.3,0,0
2,12.9,25.7,0.0,38.0,30.0,21.0,23.2,0,0
3,9.2,28.0,0.0,45.0,16.0,18.1,26.5,0,0
4,17.5,32.3,1.0,82.0,33.0,17.8,29.7,0,0


In [93]:
# Variables predictivas (o explicativas)
X = df.ix[:,(0,1,2,3,4,5,6,7)].values
# Variable objetivo
Y = df.ix[:,8].values

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [94]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state=25)

In [95]:
#Aprendizaje
LogReg = LogisticRegression()
LogReg.fit(X_train, Y_train)

/home/fabian/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [96]:
print("coeficientes del modelo: "+str(LogReg.coef_))
print("intercept: "+str(LogReg.intercept_))

coeficientes del modelo: [[ 0.0486598   0.03108154  0.01668142 -0.0062666   0.05831267 -0.02984528
  -0.02957484  0.71542363]]
intercept: [-4.6172441]


In [97]:
y_pred = LogReg.predict(X_test)

In [98]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90     32853
           1       0.73      0.40      0.52      9384

   micro avg       0.83      0.83      0.83     42237
   macro avg       0.79      0.68      0.71     42237
weighted avg       0.82      0.83      0.81     42237

